# Check `GDS` Python stack

This notebook checks all software requirements for the course Geographic Data Science are correctly installed. 

A successful run of the notebook implies no errors returned in any cell *and* every cell beyond the first one returning a printout of `True`. This ensures a correct environment installed.

In [ ]:
! echo "You are running version $GDS_ENV_VERSION of the GDS env"

## Imports

Pull list of all libs specified in the stack:

In [42]:
f = './gds_py.yml'
bespoke = {
    'h3_py': 'h3',
    'python_duckdb': 'duckdb',
    'scikit_learn': 'sklearn',
    'scikit_mobility': 'skmob',
    'xarray_spatial': 'xrspatial'
}
libs = []
with open(f, 'r') as fo:
    for l in fo.readlines():
        if ('- ' in l) and ('#' not in l) and (':' not in l):
            l = l.strip('\n').strip(' ').strip('-').strip(' ')
            l = l.split('>=')[0]
            libs.append(l)
libs = [i for i in libs if i not in [
    'conda-forge', 'libgdal-arrow-parquet', 'pip', 'cenpy', 'protobuf']
    ]
libs = [i.replace('-', '_') for i in libs]
libs = [bespoke[i] if i in bespoke else i for i in libs]

Import each library:

In [ ]:
from importlib import import_module
for lib in libs:
    print(f'Attempting to import {lib}')
    import_module(lib)

Bespoke imports:

-  `pysal`

In [ ]:
import pysal
import pysal.lib
import pysal.explore
import pysal.model
import pysal.viz

- Protobuf

In [41]:
from google.protobuf.internal import api_implementation

- `cenpy`

In [ ]:
#Issues with the Census API
import traceback
try:
    import cenpy
except Exception:
    traceback.print_exc()

## Hardcoded tests

In [ ]:
import geopandas
import geopandas as gpd
import pandas as pd

In [ ]:
from pysal import lib as libps
shp = libps.examples.get_path('columbus.shp')
db = geopandas.read_file(shp)
db.head()

In [ ]:
db[['AREA', 'PERIMETER']].to_feather('db.feather')
tst = pd.read_feather('db.feather')
! rm db.feather

In [ ]:
db.to_parquet('db.pq')
tst = gpd.read_parquet('db.pq')
! rm db.pq

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
f, ax = plt.subplots(1)
db.plot(facecolor='yellow', ax=ax)
ax.set_axis_off()
plt.show()

In [ ]:
db.crs = 'EPSG:26918'

In [ ]:
db_wgs84 = db.to_crs(epsg=4326)
db_wgs84.plot()
plt.show()

In [ ]:
from pysal.viz import splot
from splot.mapping import vba_choropleth

f, ax = vba_choropleth(db['INC'], db['HOVAL'], db)

---

In [ ]:
db.plot(column='INC', scheme='fisher_jenks', cmap=plt.matplotlib.cm.Blues)
plt.show()

In [ ]:
import osmnx
city = osmnx.geocode_to_gdf('Berkeley, California, US')
osmnx.plot_footprints(osmnx.project_gdf(city));

In [ ]:
import pyrosm
from pyrosm import get_data
# Download data for the city of Helsinki
fp = get_data("Helsinki", directory="./")
print(fp)

# Get filepath to test PBF dataset
fp = pyrosm.get_data("test_pbf")
print("Filepath to test data:", fp)

# Initialize the OSM object
osm = pyrosm.OSM(fp)

# See the type
print("Type of 'osm' instance: ", type(osm))

from pyrosm import get_data

# Pyrosm comes with a couple of test datasets
# that can be used straight away without
# downloading anything
fp = get_data("test_pbf")

# Initialize the OSM parser object
osm = pyrosm.OSM(fp)

# Read all drivable roads
# =======================
drive_net = osm.get_network(network_type="driving")
drive_net.plot()

! rm Helsinki.osm.pbf

---

In [ ]:
import numpy as np
import rasterio as rio
import contextily as ctx
tl = ctx.providers.CartoDB.Positron

db = geopandas.read_file(ps.lib.examples.get_path('us48.shp'))
db.crs = "EPSG:4326"
dbp = db.to_crs(epsg=3857)
w, s, e, n = dbp.total_bounds
# Download raster
_ = ctx.bounds2raster(w, s, e, n, 'us.tif', source=tl)
# Load up and plot
source = rio.open('us.tif', 'r')
red = source.read(1)
green = source.read(2)
blue = source.read(3)
pix = np.dstack((red, green, blue))
bounds = (source.bounds.left, source.bounds.right, \
          source.bounds.bottom, source.bounds.top)
f = plt.figure(figsize=(6, 6))
ax = plt.imshow(pix, extent=bounds)

In [ ]:
ax = db.plot()
ctx.add_basemap(ax, crs=db.crs.to_string())

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, SplitMapControl

m = Map(center=(42.6824, 365.581), zoom=5)

right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-11-11")
left_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisAquaBands721CR, "2017-11-11")

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

m

In [ ]:
# Convert us.tiff to COG with rio-cogeo
! rio cogeo create us.tif us_cog.tif
! rio cogeo validate us_cog.tif
! rio info us_cog.tif
! rio warp --dst-crs EPSG:4326 us_cog.tif us_cog_ll.tif

In [ ]:
# rioxarray + xarray_leaflet
import xarray, rioxarray
us = rioxarray.open_rasterio(
    "us_cog.tif"
).sel(
    band=1, 
    y=slice(7000000, 5000000),
    x=slice(-10000000, -8000000)
)
import xarray_leaflet
from ipyleaflet import Map

m = Map(zoom=1, basemap=basemaps.CartoDB.DarkMatter)
m

In [ ]:
l = us.astype(
    float
).rio.reproject(
    "EPSG:4326"
).leaflet.plot(m)
l.interact(opacity=(0.0,1.0))

In [ ]:
! rm us.tif us_cog.tif us_cog_ll.tif
! rm -rf cache/

In [ ]:
from IPython.display import GeoJSON

GeoJSON({
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [-118.4563712, 34.0163116]
    }
})